# Distilling Knowledge in Multiple Students  using Generative models

In [ ]:
# !nvidia-smi

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Teacher's Training

In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

print(teacher.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [ ]:
# Train the teacher model as usual
epochs = 0
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))

### Evaluation 

In [3]:
teacher.load_weights("Teacher_MNIST_98.h5")

In [4]:
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [5]:
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
# Normalize the values
X_test /= 255

In [6]:
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 2s 3ms/step - loss: 0.0484 - accuracy: 0.9874


[0.03833923488855362, 0.9894999861717224]

### Dense Vector split

In [7]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)

# 4 Students case
# ---------------------------------------------
s1Train=train_dense[:,:64]
s2Train=train_dense[:,64:128]
s3Train=train_dense[:,128:192]
s4Train=train_dense[:,192:]

## GANs' Training

In [8]:
# import np.random import random
BATCH_SIZE=32
def smooth_real_labels(y):
    return y - 0.3+(np.random.random(y.shape)*0.5)
def smooth_fake_labels(y):
    return y + (0.3 * np.random.random(y.shape))
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=input_shape)
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile(Adam(lr=0.0002),loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return gan

### Discriminator

In [9]:
def build_sdiscriminator():
    
    input2 = Input(shape=(64,),name='input')
    inp=Dense(64)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128)(leaky_relu)
    b_n = BatchNormalization()(conv3)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(256)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(512)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(1024)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)

    dense = Dense(1,activation='sigmoid',name='dense')(leaky_relu)

    output2=Dense(64)(leaky_relu)

    
    disc = Model(input2,[dense,output2])          
    disc.compile(optd,loss=['binary_crossentropy','mse'],metrics=['accuracy'])


    return disc

In [10]:
optd = Adam(lr=0.0002)
opt = Adam(lr=0.0002)

In [11]:
d1= build_sdiscriminator()
d1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 64)]         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           4160        input[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64)           0           dense_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          8320        leaky_re_lu[0][0]                
____________________________________________________________________________________________

### Generator

In [12]:
def build_sgenerator(name):

    student1 = Sequential()
    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(32, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(64,name='req'+name))

    student1.compile(opt,loss='mean_squared_error',metrics=['accuracy'])
    
    return student1

build_sgenerator('').summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 16)        2320      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 16)         

In [13]:
def training(generator,discriminator,gan,features,epo=20):
    # Setup Models here
    BATCH_SIZE = 128
    discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
        
            # Split
            inputs=X_train[index:index+BATCH_SIZE]
            sXtrain = features[index:index+BATCH_SIZE]

            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(sXtrain,[y_real,sXtrain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,sXtrain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,sXtrain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

You can initialize multiple instances here 

In [14]:
discriminator1 = build_sdiscriminator()
discriminator2 = build_sdiscriminator()
discriminator3 = build_sdiscriminator()
discriminator4 = build_sdiscriminator()


In [15]:
s1=build_sgenerator("s1")
s2=build_sgenerator('s2')
s3=build_sgenerator("s3")
s4=build_sgenerator('s4')


In [16]:
gan1 = build_gan(s1,discriminator1)
gan2 = build_gan(s2,discriminator2)
gan3 = build_gan(s3,discriminator3)
gan4 = build_gan(s4,discriminator4)

In [17]:
s1 = training(s1,discriminator1,gan1,s1Train,epo=40)
s2 = training(s2,discriminator2,gan2,s2Train,epo=35)
s3 = training(s3,discriminator3,gan3,s3Train,epo=40)
s4 = training(s4,discriminator4,gan4,s4Train,epo=35)

375/375 [==============================] - 14s 35ms/step
Epoch: 1 | Discriminator Loss: 0.650496 | Generator Loss: 2.020718 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 2 | Discriminator Loss: 0.241798 | Generator Loss: 1.187068 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 3 | Discriminator Loss: 0.183888 | Generator Loss: 1.074136 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 4 | Discriminator Loss: 0.159349 | Generator Loss: 1.023960 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 5 | Discriminator Loss: 0.146112 | Generator Loss: 0.978296 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 6 | Discriminator Loss: 0.136542 | Generator Loss: 0.915169 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 7 | Discriminator Loss: 0.129256 | Generator Loss: 0.866664 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 8 | Discriminator Loss: 0.123043 | Generator Lo

375/375 [==============================] - 13s 34ms/step
Epoch: 14 | Discriminator Loss: 0.088849 | Generator Loss: 0.577198 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 15 | Discriminator Loss: 0.087160 | Generator Loss: 0.562387 | 
375/375 [==============================] - 12s 33ms/step
Epoch: 16 | Discriminator Loss: 0.084517 | Generator Loss: 0.542401 | 
375/375 [==============================] - 12s 33ms/step
Epoch: 17 | Discriminator Loss: 0.083160 | Generator Loss: 0.524197 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 18 | Discriminator Loss: 0.081602 | Generator Loss: 0.514448 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 19 | Discriminator Loss: 0.080624 | Generator Loss: 0.505962 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 20 | Discriminator Loss: 0.079684 | Generator Loss: 0.500217 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 21 | Discriminator Loss: 0.078971 | Gene

## **4 Students**

In [13]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
o3=s3.get_layer("reqs3").output
o4=s4.get_layer("reqs4").output
output=tensorflow.keras.layers.concatenate([o1,o2,o3,o4])
print (output.shape)
output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)
mm4=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,s3.get_layer("s3").input,s4.get_layer("s4").input],output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm4.get_layer('d1').set_weights(my_weights)

(None, 256)


In [19]:
for l in mm4.layers[:len(mm4.layers)-1]:
    l.trainable=False

mm4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [20]:
batch_size = 128
mm2_history=mm4.fit([X_train,X_train,X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=70,
          verbose=1, validation_data=([X_val,X_val,X_val,X_val], Y_val))

Epoch 1/70
375/375 [==============================] - 4s 9ms/step - loss: 1.9193 - accuracy: 0.7193 - val_loss: 0.1309 - val_accuracy: 0.9753
Epoch 2/70
375/375 [==============================] - 5s 13ms/step - loss: 0.2856 - accuracy: 0.9417 - val_loss: 0.1242 - val_accuracy: 0.9773
Epoch 3/70
375/375 [==============================] - 5s 13ms/step - loss: 0.2145 - accuracy: 0.9553 - val_loss: 0.1088 - val_accuracy: 0.9779
Epoch 4/70
375/375 [==============================] - 4s 11ms/step - loss: 0.1729 - accuracy: 0.9611 - val_loss: 0.0980 - val_accuracy: 0.9780
Epoch 5/70
375/375 [==============================] - 5s 13ms/step - loss: 0.1475 - accuracy: 0.9636 - val_loss: 0.0864 - val_accuracy: 0.9789
Epoch 6/70
375/375 [==============================] - 3s 9ms/step - loss: 0.1362 - accuracy: 0.9652 - val_loss: 0.0816 - val_accuracy: 0.9790
Epoch 7/70
375/375 [==============================] - 5s 14ms/step - loss: 0.1121 - accuracy: 0.9695 - val_loss: 0.0781 - val_accuracy: 0.9803
E

375/375 [==============================] - 4s 10ms/step - loss: 0.0980 - accuracy: 0.9713 - val_loss: 0.0700 - val_accuracy: 0.9797
Epoch 59/70
375/375 [==============================] - 4s 11ms/step - loss: 0.0984 - accuracy: 0.9721 - val_loss: 0.0701 - val_accuracy: 0.9795
Epoch 60/70
375/375 [==============================] - 5s 13ms/step - loss: 0.1014 - accuracy: 0.9707 - val_loss: 0.0701 - val_accuracy: 0.9796
Epoch 61/70
375/375 [==============================] - 5s 13ms/step - loss: 0.1095 - accuracy: 0.9682 - val_loss: 0.0704 - val_accuracy: 0.9801
Epoch 62/70
375/375 [==============================] - 4s 12ms/step - loss: 0.1026 - accuracy: 0.9706 - val_loss: 0.0718 - val_accuracy: 0.9797
Epoch 63/70
375/375 [==============================] - 5s 14ms/step - loss: 0.1035 - accuracy: 0.9704 - val_loss: 0.0705 - val_accuracy: 0.9787
Epoch 64/70
375/375 [==============================] - 4s 10ms/step - loss: 0.1039 - accuracy: 0.9707 - val_loss: 0.0712 - val_accuracy: 0.9789
Epoc

In [21]:
loss, acc = mm4.evaluate([X_test,X_test,X_test,X_test], Y_test, verbose=1)
loss, acc 

313/313 [==============================] - 3s 9ms/step - loss: 6.8226 - accuracy: 0.9837


(6.822558403015137, 0.9836999773979187)